# Packages and Files

In [1]:
import os
import sys

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import PIL
import torch
import torchvision
from PIL import Image
from torchvision import transforms

In [2]:
print("Platform:", sys.platform)
print("Python version:", sys.version)
print("---")
print("matplotlib version:", matplotlib.__version__)
print("pandas version:", pd.__version__)
print("PIL version:", PIL.__version__)
print("torch version:", torch.__version__)
print("torchvision version:", torchvision.__version__)

Platform: win32
Python version: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]
---
matplotlib version: 3.10.8
pandas version: 2.3.3
PIL version: 12.0.0
torch version: 2.9.1+cpu
torchvision version: 0.24.1+cpu


# Explore Files

In [3]:
data_dir = os.path.join("data_p1", "data_multiclass")
train_dir = os.path.join(data_dir, "train")

print("data_dir class:", type(data_dir))
print("Data directory:", data_dir)
print()
print("train_dir class:", type(train_dir))
print("Training data directory:", train_dir)

data_dir class: <class 'str'>
Data directory: data_p1\data_multiclass

train_dir class: <class 'str'>
Training data directory: data_p1\data_multiclass\train


In [4]:
class_directories = os.listdir(train_dir)

print("class_directories type:", type(class_directories))
print("class_directories length:", len(class_directories))
print(class_directories)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data_p1\\data_multiclass\\train'